In [5]:
import warnings
warnings.filterwarnings("ignore")

import tkinter as tk
from tkinter import Frame, Label, Scrollbar, Canvas
from PIL import Image, ImageTk
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

class ChatBotApp:
    def __init__(self, root):
        self.root = root
        self.root.title("ChatBot")
        self.root.geometry("500x700")

        self.create_widgets()
        
        # Load GPT-2 tokenizer and model
        self.gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
        self.gpt2_model = GPT2LMHeadModel.from_pretrained('gpt2')

    def create_widgets(self):
        # Load background image
        self.bg_image = Image.open("background.jpg")
        self.bg_image = self.bg_image.resize((500, 700), Image.LANCZOS)
        self.bg_photo = ImageTk.PhotoImage(self.bg_image)

        # Create a label to display the background image
        self.bg_label = Label(self.root, image=self.bg_photo)
        self.bg_label.place(relwidth=1, relheight=1)

        # Create a frame for the conversation display area
        self.conversation_frame = Frame(self.bg_label, bg='#EAEAEA')
        self.conversation_frame.pack(padx=10, pady=10, fill=tk.BOTH, expand=True)

        # Create a canvas to allow scrolling
        self.canvas = Canvas(self.conversation_frame, bg='#EAEAEA', bd=0, highlightthickness=0)
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=True)

        self.scrollbar = Scrollbar(self.conversation_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.canvas.bind('<Configure>', lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))

        self.message_frame = Frame(self.canvas, bg='#EAEAEA')
        self.canvas.create_window((0, 0), window=self.message_frame, anchor="nw")

        # Create a frame for the entry and send button
        entry_frame = Frame(self.bg_label, bg='#EAEAEA')
        entry_frame.pack(pady=10, fill=tk.X)

        # Create an entry widget for user input
        self.user_input = tk.Entry(entry_frame, width=50, font=('Arial', 12))
        self.user_input.pack(side=tk.LEFT, padx=(0, 10), fill=tk.X, expand=True)

        # Create a send button
        send_button = tk.Button(entry_frame, text="Send", command=self.send_message, bg='#4CAF50', fg='white', font=('Arial', 10))
        send_button.pack(side=tk.RIGHT)

    def send_message(self):
        user_message = self.user_input.get().strip()
        if user_message:
            self.display_message("You", user_message, "right")
            bot_response = self.get_bot_response(user_message)
            self.display_message("Bot", bot_response, "left")
            self.user_input.delete(0, tk.END)

    def display_message(self, sender, message, side):
        sender_font = ('Arial', 12, 'bold')
        message_font = ('Arial', 12)

        sender_label = Label(self.message_frame, text=f"{sender}:", bg='#FFFFFF', wraplength=350, justify='left', anchor='w' if side == 'left' else 'e', padx=10, pady=5, font=sender_font)
        sender_label.pack(padx=10, pady=5, anchor='w' if side == 'left' else 'e')

        message_label = Label(self.message_frame, text=message, bg='#FFFFFF', wraplength=350, justify='left', anchor='w' if side == 'left' else 'e', padx=10, pady=5, font=message_font)
        message_label.pack(padx=10, pady=(0, 5), anchor='w' if side == 'left' else 'e')

        self.canvas.update_idletasks()
        self.canvas.configure(scrollregion=self.canvas.bbox("all"))
        self.canvas.yview_moveto(1.0)

    def get_bot_response(self, user_message):
        # Generate response using GPT-2
        inputs = self.gpt2_tokenizer.encode(user_message, return_tensors='pt')
        outputs = self.gpt2_model.generate(inputs, max_new_tokens=50, num_return_sequences=1, pad_token_id=self.gpt2_tokenizer.eos_token_id)
        response = self.gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        # Ensure response is properly formatted
        response = response.split('.')[0] + '.'
        return response

root = tk.Tk()
app = ChatBotApp(root)
root.mainloop()
